## Blog Assignment

For our project we want to merge the following three datasets:

- Netflix Originals Movies with IMDB scores 

- General Netflix Series / Movies data  (From this we want to add the rating column to our Dataframe)

- Netflix Stockprices 

With the merged Dataframe we'll do extensive data analysis with help from Pandas for data manipulation and Plotly to make interactive visualisations.
Our goal is to discover if there is any corralation between the different features in this dataset. Examples of this can be corralation with movie releases and stock prices, IMDB scores and stock prices, rating and IMDB score.

### Using the Kaggle API to download the datasets we want to use during our project

The first step is to install the Kaggle library, and use it download the datasets.

- Documentation link: https://www.kaggle.com/docs/api
- Code source: https://www.youtube.com/watch?v=DgGFhQmfxHo&t=331s

In [13]:
# install the Kaggle library 
!pip install kaggle

  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73049 sha256=bf889e8369bf17bdee997940f61af38d6b1fc79105646bd16316d41e5db86c9e
  Stored in directory: c:\users\gebruiker\appdata\local\pip\cache\wheels\ac\b2\c3\fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle


Kaggle requires that the authentication key is present on our device ( using the following path: ~/.kaggle/kaggle.json), so we'll have to make a directory before we can proceed any futher

In [38]:
# Try to import the Kaggle library. Before proceeding create the directory specified below, and place the kaggle.json there
import kaggle

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [39]:
# Try to import the Kaggle library again
import kaggle

In [40]:
# Import the Kaggle API from the Kaggle library
from kaggle.api.kaggle_api_extended import KaggleApi

In [41]:
# instantiate the API, then authenticate (uses the kaggle.json for login credentials)
api = KaggleApi()
api.authenticate()

print("Succesfully connected to the Kaggle API!")

Succesfully connected to the Kaggle API!


In [42]:
# Use the Kaggle API to download the datasets we're using during the project
api.dataset_download_file("luiscorter/netflix-original-films-imdb-scores",
file_name="NetflixOriginals.csv")

api.dataset_download_file("ariyoomotade/netflix-data-cleaning-analysis-and-visualization",
file_name="netflix1.csv")

api.dataset_download_file("akpmpr/updated-netflix-stock-price-all-time",
file_name="netflix.csv")

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Length': '0', 'Date': 'Tue, 27 Sep 2022 07:57:38 GMT', 'Access-Control-Allow-Credentials': 'true', 'Set-Cookie': 'ka_sessionid=398f32e211b231df23a997d72eb1035e; max-age=2626560; path=/, GCLB=CJi2rd6S_oi5kwE; path=/; HttpOnly', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/datasets/download/luiscorter/netflix-original-films-imdb-scores/NetflixOriginals.csv', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-kqx/Ru4KIml5TH9I82XBVA==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ https://polygraph-cool.github.io https://www.google.com/recaptcha/ https://form.jotform.com https://submit.jotform.us https://submit.jotformpro.com https://submit.jotform.com https://www.docdroid.com https://www.docd

### Dealing with encoded CSV files

Before we can proceed with loading the CSV files with Pandas there is one more step we need to take.
The CSV files are encoded, and we'll use the chardet library to discover what type of encoding it is.
After which we'll use the `encoding parameter` of Pandas to properly load the CSV file.

In [167]:
# Import the needed library
import chardet

# Create a dict with file paths
files = {"NetflixOriginals.csv": "./data/NetflixOriginals.csv", "netflix.csv": "./data/netflix.csv", "netflix1.csv": "./data/netflix1.csv"}

# Loop through the dict, and print out the names and encoding type 
for name, file in files.items():
    with open(file, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100000))
    print(name, result)

NetflixOriginals.csv {'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
netflix.csv {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
netflix1.csv {'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


### Loading in and Merging the Dataframes

Now we can start loading the datasets into Pandas with the `read_csv` function, and we will use the `merge` function to join the dataframes together.

- Note: It's only mandatory to use the encoding parameter for the Windows-1252 enconding, because Pandas doesn't have problems with loading in the other encoding formats

In [127]:
# Import Pandas
import pandas as pd

In [151]:
# Import the first dataset, and decode the csv file with Windows-1252 encoding
df = pd.read_csv("./data/NetflixOriginals.csv", encoding="Windows-1252")

# Change the column name of Premiere and set the format in DateTime to match the other dataset 
df.rename(columns = {"Premiere": "Date"}, inplace=True)
df.rename(columns = {"Title": "title"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"])

# Show the first 5 rows
df.head()

,title,Genre,Date,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,2019-08-05,58,2.5,English/Japanese
1,Dark Forces,Thriller,2020-08-21,81,2.6,Spanish
2,The App,Science fiction/Drama,2019-12-26,79,2.6,Italian
3,The Open House,Horror thriller,2018-01-19,94,3.2,English
4,Kaali Khuhi,Mystery,2020-10-30,90,3.4,Hindi


In [152]:
# Load in the second dataset
df2 = pd.read_csv("./data/netflix1.csv")

# Change the column name of title to match the other dataset
# df2.rename(columns = {"title": "Title"}, inplace=True)

# Show the first 5 rows
df2.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"


In [153]:
# Load in the third dataset
df3 = pd.read_csv("./data/netflix.csv")

df3["Date"] = pd.to_datetime(df3["Date"])

# Show the first 5 rows
df3.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2002-05-23,1.242857,1.145714,1.156429,1.196429,104790000.0,1.196429
1,2002-05-24,1.225000,1.197143,1.214286,1.210000,11104800.0,1.210000
2,2002-05-28,1.232143,1.157143,1.213571,1.157143,6609400.0,1.157143
3,2002-05-29,1.164286,1.085714,1.164286,1.103571,6757800.0,1.103571
4,2002-05-30,1.107857,1.071429,1.107857,1.071429,10154200.0,1.071429


When merging Dataframes there are multiple options on how to do it:

- Inner: keep all rows from both Dataframes that match (Which can result in dataloss in our case)

- Outer: keep all rows from both Dataframes (Which results in alot of rows with NaN values in our case)

- Left: Include all rows from Dataframe X and only those from y that match (X= target Dataframe, y= Dataframe to merge in)

- Right: Inculde all from Dataframe y and only those from X that match

We've made the the choice to use a Left Join, because Dataframe X is alot shorter then y

In [154]:
# Create a new dataframe by merging df, df2, df3 on shared column names and using the Left Join
netflix_df = df.merge(df2[["title", "rating"]], on="title", how="left") \
        .merge(df3, on="Date", how="left") 
netflix_df.head()

,title,Genre,Date,Runtime,IMDB Score,Language,rating,High,Low,Open,Close,Volume,Adj Close
0,Enter the Anime,Documentary,2019-08-05,58,2.5,English/Japanese,TV-MA,313.420013,304.679993,310.959991,307.630005,8692500.0,307.630005
1,Dark Forces,Thriller,2020-08-21,81,2.6,Spanish,TV-MA,498.130005,490.000000,496.459991,492.309998,3921300.0,492.309998
2,The App,Science fiction/Drama,2019-12-26,79,2.6,Italian,TV-MA,336.459991,332.010010,334.600006,332.630005,3589900.0,332.630005
3,The Open House,Horror thriller,2018-01-19,94,3.2,English,TV-MA,223.490005,218.500000,222.750000,220.460007,10548600.0,220.460007
4,Kaali Khuhi,Mystery,2020-10-30,90,3.4,Hindi,TV-14,505.880005,472.209991,502.010010,475.739990,7807900.0,475.739990


## Data Cleaning

In [155]:
# Check the number of rows and datatypes
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 584 entries, 0 to 583
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       584 non-null    object        
 1   Genre       584 non-null    object        
 2   Date        584 non-null    datetime64[ns]
 3   Runtime     584 non-null    int64         
 4   IMDB Score  584 non-null    float64       
 5   Language    584 non-null    object        
 6   rating      504 non-null    object        
 7   High        548 non-null    float64       
 8   Low         548 non-null    float64       
 9   Open        548 non-null    float64       
 10  Close       548 non-null    float64       
 11  Volume      548 non-null    float64       
 12  Adj Close   548 non-null    float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(4)
memory usage: 63.9+ KB


In [156]:
# Check if there are NaN values
netflix_df.isna().sum()

title          0
Genre          0
Date           0
Runtime        0
IMDB Score     0
Language       0
rating        80
High          36
Low           36
Open          36
Close         36
Volume        36
Adj Close     36
dtype: int64

The Rating column and all the columns with Stock data have NaN values. We now have to decide what we want to do with the those rows, there are two main options for this: `pd.fillna` or `pd.dropna`. 

##### Rating column
For the Rating column we have decided to fill all NaN values with U for unknown, because 80 rows would be alot of data to discard.

##### Stock data columns
For the Stock data columns we've tried to apply the `pd.interpolate` function, but it wasn't succesfull . We've decided to drop the rows in question, because dropping 36 rows could be better justified

In [162]:
# Sort all values in the Dataframe by date, to order the data first
netflix_df.sort_values("Date", inplace=True)

# Fill the NaN values of the rating column with U 
netflix_df["rating"].fillna("U", inplace=True)

# Drop all rows with NaN values
netflix_df.dropna(inplace=True)

In [165]:
# Check the number of rows and datatypes again
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 548 entries, 519 to 369
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       548 non-null    object        
 1   Genre       548 non-null    object        
 2   Date        548 non-null    datetime64[ns]
 3   Runtime     548 non-null    int64         
 4   IMDB Score  548 non-null    float64       
 5   Language    548 non-null    object        
 6   rating      548 non-null    object        
 7   High        548 non-null    float64       
 8   Low         548 non-null    float64       
 9   Open        548 non-null    float64       
 10  Close       548 non-null    float64       
 11  Volume      548 non-null    float64       
 12  Adj Close   548 non-null    float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(4)
memory usage: 59.9+ KB


In [163]:
# Check the number of NaN values again
netflix_df.isna().sum()

title         0
Genre         0
Date          0
Runtime       0
IMDB Score    0
Language      0
rating        0
High          0
Low           0
Open          0
Close         0
Volume        0
Adj Close     0
dtype: int64

In [164]:
# Show the first 5 rows
netflix_df.head()

,title,Genre,Date,Runtime,IMDB Score,Language,rating,High,Low,Open,Close,Volume,Adj Close
519,The Other One: The Long Strange Trip of Bob Weir,Documentary,2015-05-22,83,7.3,English,TV-14,89.407143,88.692856,89.178574,88.838570,7466200.0,88.838570
228,Hot Girls Wanted,Documentary,2015-05-29,84,6.1,English,TV-MA,90.205711,88.955711,89.472855,89.151428,12712000.0,89.151428
550,"What Happened, Miss Simone?",Documentary,2015-06-26,84,7.6,English,TV-14,95.314285,93.088570,95.300003,93.088570,30314900.0,93.088570
531,Tig,Documentary,2015-07-17,80,7.4,English,TV-14,117.879997,114.239998,117.339996,114.769997,25136900.0,114.769997
465,Keith Richards: Under the Influence,Documentary,2015-09-18,81,7.1,English,TV-14,104.360001,100.709999,102.000000,102.620003,21715000.0,102.620003
